In [ ]:
# Load Churn_Modeling.csv into a pandas DataFrame

import pandas as pd

df = pd.read_csv('Churn_Modelling.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [ ]:
df.describe()

In [ ]:
# Convert negative EstimatedSalary values to positive

df['EstimatedSalary'] = df['EstimatedSalary'].abs()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()

In [ ]:
df.describe()

In [ ]:
# Cap credit score at 900
df['CreditScore'] = df['CreditScore'].clip(upper=900)

In [ ]:
df.describe()

In [ ]:
# Convert categorical Geography to numerical using label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Geography'] = le.fit_transform(df['Geography'])
df['Gender'] = le.fit_transform(df['Gender'])
df.head()


In [ ]:
# Plot feature importance
import matplotlib.pyplot as plt
import seaborn as sns
feature_importance = df.corr()['Exited'].abs().sort_values(ascending=False)
plt.figure(figsize=(10,6))
sns.barplot(x=feature_importance.index, y=feature_importance.values)
plt.xticks(rotation=45)
plt.title('Feature Importance for Churn Prediction')
plt.show()

In [ ]:
# Create decision tree model to predict churn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
X = df.drop(['HasCrCard', 'EstimatedSalary', 'Tenure', 'Exited', 'CreditScore', 'Geography'], axis=1)
y = df['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = DecisionTreeClassifier(random_state=42, max_depth=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# Plot the decision tree
from sklearn.tree import plot_tree
plt.figure(figsize=(200,100))
plot_tree(model, filled=True, feature_names=X.columns, class_names=['Stay', 'Exit'])
plt.show()

In [ ]:
# Hyper parameter tuning using GridSearchCV

from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [None, 3, 5, 7, 10],
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))
print("Best Hyperparameters:", grid_search.best_estimator_)


In [ ]:
# Random Forest Classifier for churn prediction
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

In [ ]:
# Grid Search for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 20]
}
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=2)
grid_search_rf.fit(X_train, y_train)
best_rf_model = grid_search_rf.best_estimator_
y_pred_best_rf = best_rf_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_best_rf))
print(classification_report(y_test, y_pred_best_rf))

In [ ]:
# Logistic regression model to predict Outcome and evaluate accuracy
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

In [ ]:
# Grid Search for Logistic Regression
param_grid_log = {
    'C': [0.01, 0.1, 1, 10
    ],
    'solver': ['liblinear', 'saga']
}

grid_search_log = GridSearchCV(estimator=log_model, param_grid=param_grid_log, cv=5, n_jobs=-1, verbose=2)
grid_search_log.fit(X_train, y_train)
best_log_model = grid_search_log.best_estimator_
y_pred_best_log = best_log_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_best_log))
print(classification_report(y_test, y_pred_best_log))

In [ ]:
# Plot feature coefficients for logistic regression
plt.figure(figsize=(10, 6))
coefficients = best_log_model.coef_[0]
feature_names = X.columns
plt.barh(feature_names, coefficients)
plt.xlabel('Coefficient Value')
plt.title('Logistic Regression Feature Coefficients')
plt.tight_layout()
plt.show()

# Alternative: Plot prediction probabilities distribution
y_prob = best_log_model.predict_proba(X_test)[:, 1]
plt.figure(figsize=(10, 6))
plt.hist(y_prob[y_test == 0], alpha=0.7, label='Stay', bins=30)
plt.hist(y_prob[y_test == 1], alpha=0.7, label='Exit', bins=30)
plt.xlabel('Predicted Probability of Churn')
plt.ylabel('Frequency')
plt.title('Distribution of Predicted Probabilities')
plt.legend()
plt.show()


In [ ]:
# Save the best model using joblib
import joblib
joblib.dump(best_log_model, 'best_logistic_regression_model.joblib')